In [196]:
import sympy
from sympy import *
from sympy.vector import *
from sympy import symbols
D = Derivative

In [73]:
init_session()

IPython console for SymPy 1.13.3 (Python 3.13.3-64-bit) (ground types: python)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.13.3/



In [3]:
sympy.sqrt(8)

2⋅√2

In [4]:
x, y = symbols('x y')
expr = x + 2 * y
expr

x + 2⋅y

In [5]:
diff(cos(x) * exp(x), x)

   x           x       
- ℯ ⋅sin(x) + ℯ ⋅cos(x)

In [339]:
u = Function('u')(x, t)
v = Function('v')(x, t)
laplacian = Symbol('Δ')
phi = Function('φ')(x)
f = Function('f')(x, t)
c = symbols('c')

wave_eq = diff(u, t, 2) - c**2 * laplacian * u - f
print("Base equation: ")
Eq(wave_eq, 0)

Base equation: 


                            2              
   2                       ∂               
- c ⋅Δ⋅u(x, t) - f(x, t) + ───(u(x, t)) = 0
                             2             
                           ∂t              

Réarrangons les termes, avec les fonctions inconnues à gauche et les connues à droite.

In [340]:
terms = Add.make_args(wave_eq)

terms_with_u = [term for term in terms if term.has(u) or term.has(laplacian_u)]
terms_without_u = [term for term in terms if not term.has(u) or term.has(laplacian_u)]

lhs = Add(*terms_with_u)
rhs = -Add(*terms_without_u)
Eq(lhs, rhs)

                  2                    
   2             ∂                     
- c ⋅Δ⋅u(x, t) + ───(u(x, t)) = f(x, t)
                   2                   
                 ∂t                    

Introduisont la variable $v = \frac{\partial u}{\partial t}$ afin de remplacer les dérivées secondes en temps en dérivée premières en temps.

Cela permettra de simplifier l'écriture du schéma.

In [341]:
lhs = lhs.subs(diff(u, t, 2), diff(v, t))
v_eq = diff(u, t) - v
[Eq(lhs, rhs), Eq(v_eq, 0)]

⎡   2             ∂                                 ∂              ⎤
⎢- c ⋅Δ⋅u(x, t) + ──(v(x, t)) = f(x, t), -v(x, t) + ──(u(x, t)) = 0⎥
⎣                 ∂t                                ∂t             ⎦

In [342]:
order = ode_order(lhs, v)
order

1

In [343]:
var_eq = Eq(Integral(lhs * v, x), Integral(rhs * v,x))
var_eq

⌠                                                                 
⎮ ⎛   2             ∂          ⎞              ⌠                   
⎮ ⎜- c ⋅Δ⋅u(x, t) + ──(v(x, t))⎟⋅v(x, t) dx = ⎮ f(x, t)⋅v(x, t) dx
⎮ ⎝                 ∂t         ⎠              ⌡                   
⌡                                                                 

### Time discretization

In [344]:
theta = symbols('theta')
t_curr, t_prev = symbols('t_n t_n-1')
u_curr, u_prev = symbols('u^n u^n-1')
v_curr, v_prev = symbols('v^n v^n-1')
f_curr, f_prev = symbols('f^n f^n-1')

k = symbols('k')# t_curr - t_prev # length of current time step
exprs = (lhs, rhs, v_eq)
res = [expr.subs(
    diff(u, t),
    (u_curr - u_prev) / k
).subs(
    diff(v, t),
    (v_curr - v_prev) / k
)
.subs(
    f,
    theta * f_curr + (1 - theta) * f_prev
)
.subs(
    v,
    theta * v_curr + (1 - theta) * v_prev
)
.subs(
    u,
    theta * u_curr + (1 - theta) * u_prev
)
       for expr in exprs]
# res = [expr.subs(theta, 0) for expr in res]
lhs_d, rhs_d, v_eq_d = res

Première équation discretisée :

In [345]:
eq_1_d = Eq(v_eq_d, 0)
eq_1_d

                       uⁿ - uⁿ⁻¹    
-θ⋅vⁿ - vⁿ⁻¹⋅(1 - θ) + ───────── = 0
                           k        

Seconde équation discretisée :

In [346]:
eq_2_d = Eq(lhs_d, rhs_d)
eq_2_d

   2                           vⁿ - vⁿ⁻¹                      
- c ⋅Δ⋅(θ⋅uⁿ + uⁿ⁻¹⋅(1 - θ)) + ───────── = fⁿ⋅θ + fⁿ⁻¹⋅(1 - θ)
                                   k                          

**Crank-Nicolson** ($\theta = \frac{1}{2}$)

In [347]:
eq_1_d.subs(theta, Rational(1, 2))

  vⁿ   vⁿ⁻¹   uⁿ - uⁿ⁻¹    
- ── - ──── + ───────── = 0
  2     2         k        

In [348]:
simplify(Laplacian(u + v))

0

In [336]:
simplify(D(u, x, 2) + D(u, y, 2))

 2          
∂           
───(u(x, t))
  2         
∂x          

In [381]:
def mult_eq(eq, expr):
    return Eq(eq.lhs * expr, eq.rhs * expr)

a = eq_1_d.subs(v_curr, solve(eq_2_d, v_curr)[0])
eq_1_d_s = Eq(u_curr, solve(a, u_curr)[0])
eq_1_d_s = mult_eq(eq_1_d_s, denom(eq_1_d_s.rhs))
eq_1_d_s

   ⎛ 2  2  2      ⎞    2  2  2           2  2                2  2         2  2 ↪
uⁿ⋅⎝c ⋅k ⋅θ ⋅Δ - 1⎠ = c ⋅k ⋅θ ⋅uⁿ⁻¹⋅Δ - c ⋅k ⋅θ⋅uⁿ⁻¹⋅Δ - fⁿ⋅k ⋅θ  + fⁿ⁻¹⋅k ⋅θ  ↪

↪          2                  
↪  - fⁿ⁻¹⋅k ⋅θ - k⋅vⁿ⁻¹ - uⁿ⁻¹

In [367]:
eq_2_d_s = Eq(v_curr, solve(eq_2_d, v_curr)[0])
eq_2_d_s

      2             2               2                                          ↪
vⁿ = c ⋅k⋅θ⋅uⁿ⋅Δ - c ⋅k⋅θ⋅uⁿ⁻¹⋅Δ + c ⋅k⋅uⁿ⁻¹⋅Δ + fⁿ⋅k⋅θ - fⁿ⁻¹⋅k⋅θ + fⁿ⁻¹⋅k +  ↪

↪     
↪ vⁿ⁻¹

In [390]:


def integ_eq(eq):
    return Eq(Integral(eq.lhs, x), Integral(eq.rhs, x))

def variational(eq):
    return integ_eq(mult_eq(eq, phi))

eq_1_v = variational(eq_1_d_s)
eq_1_v

⌠                               ⌠                                              ↪
⎮    ⎛ 2  2  2      ⎞           ⎮ ⎛ 2  2  2           2  2                2  2 ↪
⎮ uⁿ⋅⎝c ⋅k ⋅θ ⋅Δ - 1⎠⋅φ(x) dx = ⎮ ⎝c ⋅k ⋅θ ⋅uⁿ⁻¹⋅Δ - c ⋅k ⋅θ⋅uⁿ⁻¹⋅Δ - fⁿ⋅k ⋅θ  ↪
⌡                               ⌡                                              ↪

↪                                                   
↪          2  2         2                  ⎞        
↪  + fⁿ⁻¹⋅k ⋅θ  - fⁿ⁻¹⋅k ⋅θ - k⋅vⁿ⁻¹ - uⁿ⁻¹⎠⋅φ(x) dx
↪                                                   

In [391]:
eq_2_v = variational(eq_2_d_s)
eq_2_v

               ⌠                                                               ↪
⌠              ⎮ ⎛ 2             2               2                             ↪
⎮ vⁿ⋅φ(x) dx = ⎮ ⎝c ⋅k⋅θ⋅uⁿ⋅Δ - c ⋅k⋅θ⋅uⁿ⁻¹⋅Δ + c ⋅k⋅uⁿ⁻¹⋅Δ + fⁿ⋅k⋅θ - fⁿ⁻¹⋅k⋅ ↪
⌡              ⌡                                                               ↪

↪                           
↪                  ⎞        
↪ θ + fⁿ⁻¹⋅k + vⁿ⁻¹⎠⋅φ(x) dx
↪                           